In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
import torch
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import json

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

train_path = './aggregated_7_480/'
output_path = './prepared_data/'

if not os.path.exists(output_path):
    os.makedirs(output_path)

MAX_Q_SENSORS = 10

In [ ]:
def compute_daily_pattern(data, minutes_per_day=1440):
    daily_data = data[:minutes_per_day].mean(axis=1)
    return daily_data.std()

def compute_weekly_pattern(data, minutes_per_week=10080):
    if len(data) >= minutes_per_week:
        weekly_data = data[:minutes_per_week].mean(axis=1)
        return weekly_data.std()
    return 0

def calculate_global_stats(path):
    q_values = []
    p_values = []
    m_values = []

    file_list = glob(os.path.join(path, '*.csv'))
    for file in file_list:
        df = pd.read_csv(file)
        q_sensors = [col for col in df.columns if col.startswith('Q')]
        p_sensors = [col for col in df.columns if col.startswith('P') and not col.endswith('_flag')]

        q_values.extend(df[q_sensors].values.flatten())
        p_values.extend(df[p_sensors].values.flatten())
        m_sum = df[[col for col in df.columns if col.startswith('M')]].sum(axis=1)
        m_values.extend(m_sum.values)

    q_mean, q_std = np.mean(q_values), np.std(q_values)
    p_mean, p_std = np.mean(p_values), np.std(p_values)
    m_mean, m_std = np.mean(m_values), np.std(m_values)

    return {
        "Q": {"mean": q_mean, "std": q_std},
        "P": {"mean": p_mean, "std": p_std},
        "M_sum": {"mean": m_mean, "std": m_std}
    }

def normalize_and_compute_features(df, max_q_sensors=MAX_Q_SENSORS):
    # 각 칼럼별로 Z-Score 정규화
    normalized_df = df.copy()
    
    q_sensors = [col for col in df.columns if col.startswith('Q')]
    p_sensors = [col for col in df.columns if col.startswith('P') and not col.endswith('_flag')]
    m_columns = [col for col in df.columns if col.startswith('M')]
    
    # 각 센서 타입별로 정규화
    for sensor in q_sensors + p_sensors + m_columns:
        normalized_df[sensor] = (df[sensor] - df[sensor].mean()) / (df[sensor].std() + 1e-8)
    
    # 정규화된 데이터로 피처 계산
    padded_q_data = np.zeros((len(df), max_q_sensors))
    
    for idx, sensor in enumerate(q_sensors):
        if idx >= max_q_sensors:
            break
        padded_q_data[:, idx] = normalized_df[sensor]
    
    m_sum = normalized_df[m_columns].sum(axis=1)

    features = {
        'mean': np.mean(padded_q_data, axis=0),
        'std': np.std(padded_q_data, axis=0),
        'skewness': pd.DataFrame(padded_q_data).skew().values,
        'kurtosis': pd.DataFrame(padded_q_data).kurtosis().values,
        'm_stats': {
            'mean': np.mean(m_sum),
            'std': np.std(m_sum),
            'skewness': pd.Series(m_sum).skew(),
            'kurtosis': pd.Series(m_sum).kurtosis()
        }
    }
    
    features.update({
        'temporal': {
            'autocorr': pd.Series(padded_q_data.mean(axis=1)).autocorr(),
            'trend': np.polyfit(np.arange(len(padded_q_data)), 
                              padded_q_data.mean(axis=1), 1)[0],
            'std_diff': np.std(np.diff(padded_q_data, axis=0)),
            'daily_pattern': compute_daily_pattern(padded_q_data),
            'weekly_pattern': compute_weekly_pattern(padded_q_data)
        }
    })
    
    return padded_q_data, m_sum, features

In [ ]:
class TimeSeriesSensorDataset(Dataset):
    def __init__(self, data, window_size=1440, stride=720):  # 하루 단위 윈도우, 12시간 스트라이드
        """
        Args:
            data: numpy array of shape [n_samples, n_features]
            window_size: number of time steps in each window (default: 1440 minutes = 1 day)
            stride: number of time steps to move forward (default: 720 minutes = 12 hours)
        """
        self.data = torch.tensor(data, dtype=torch.float32)
        self.window_size = window_size
        self.stride = stride
    
    def __len__(self):
        return (len(self.data) - self.window_size) // self.stride + 1
    
    def __getitem__(self, idx):
        start_idx = idx * self.stride
        window = self.data[start_idx:start_idx + self.window_size]
        # 목표는 윈도우의 마지막 시점
        target = self.data[start_idx + self.window_size - 1]
        return window, target

class TimeSeriesSensorAutoencoder(torch.nn.Module):
    def __init__(self, input_dim, latent_dim=32, hidden_dim=256, window_size=1440, weight_decay=1e-4, sensor_weights=None):
        super(TimeSeriesSensorAutoencoder, self).__init__()
        self.weight_decay = weight_decay
        
        if sensor_weights is None:
            self.sensor_weights = {
                'Q': 1.0,
                'M': 0.5,
                'P': 2.0
            }
        else:
            self.sensor_weights = sensor_weights
            
        # 시계열 인코더 (LSTM 사용)
        self.encoder_lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )
        
        # LSTM 출력을 latent space로 매핑
        self.encoder_fc = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, hidden_dim),  # *2는 bidirectional 때문
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(hidden_dim, latent_dim),
            torch.nn.BatchNorm1d(latent_dim),
            torch.nn.ReLU()
        )
        
        # latent vector를 시계열 길이로 확장
        self.decoder_fc = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3)
        )
        
        # 시계열 디코더
        self.decoder_lstm = torch.nn.LSTM(
            input_size=hidden_dim,
            hidden_size=hidden_dim,
            num_layers=2,
            batch_first=True,
            dropout=0.3,
            bidirectional=True
        )
        
        # 최종 출력층
        self.output_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_dim * 2, hidden_dim),
            torch.nn.BatchNorm1d(hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(hidden_dim, input_dim)
        )
    
    def forward(self, x):
        batch_size = x.size(0)
        device = x.device
        
        # 인코딩
        lstm_out, _ = self.encoder_lstm(x)
        # 마지막 시점의 출력만 사용
        last_hidden = lstm_out[:, -1, :]
        z = self.encoder_fc(last_hidden)
        
        # 디코딩
        decoded = self.decoder_fc(z)
        decoded = decoded.unsqueeze(1).repeat(1, x.size(1), 1)
        decoded, _ = self.decoder_lstm(decoded)
        reconstructed = self.output_layer(decoded[:, -1, :])  # 마지막 시점만 예측
        
        # 마스크 생성 (마지막 시점에 대해서만)
        q_mask = torch.zeros((batch_size, x.size(-1)), device=device)
        q_mask[:, :MAX_Q_SENSORS] = (x[:, -1, :MAX_Q_SENSORS] != 0).float()
        
        m_mask = torch.zeros((batch_size, x.size(-1)), device=device)
        m_mask[:, -2] = 1
        
        p_mask = torch.zeros((batch_size, x.size(-1)), device=device)
        p_mask[:, -1] = 1
        
        # 손실 계산 (마지막 시점에 대해서만)
        target = x[:, -1, :]
        
        q_loss = torch.nn.functional.mse_loss(
            reconstructed * q_mask,
            target * q_mask,
            reduction='sum'
        ) / (q_mask.sum() + 1e-8)
        
        m_loss = torch.nn.functional.mse_loss(
            reconstructed * m_mask,
            target * m_mask,
            reduction='mean'
        )
        
        p_loss = torch.nn.functional.mse_loss(
            reconstructed * p_mask,
            target * p_mask,
            reduction='mean'
        )
        
        total_loss = (
            self.sensor_weights['Q'] * q_loss + 
            self.sensor_weights['M'] * m_loss + 
            self.sensor_weights['P'] * p_loss + 
            self.weight_decay * sum(p.norm(2) ** 2 for p in self.parameters())
        )
        
        return z, reconstructed, total_loss

class SensorClusterManager:
    def __init__(self, n_clusters=6):
        self.n_clusters = n_clusters
        self.kmeans = KMeans(n_clusters=n_clusters)
        self.cluster_models = {}
        self.cluster_stats = {}
        self.sensor_weights = {
            'Q': 1.0,
            'M': 0.5,
            'P': 2.0
        }
        
    def compute_cluster_assignment(self, sensor_data_dict):
        p_values = []
        sensor_ids = list(sensor_data_dict.keys())
        
        for sensor_id in sensor_ids:
            # timestamp 열을 제외한 숫자 데이터만 사용
            numeric_columns = sensor_data_dict[sensor_id].columns[:-1]  # timestamp 열 제외
            p_mean = np.mean(sensor_data_dict[sensor_id][numeric_columns].values[:, -1])
            p_values.append(p_mean)
        
        p_values = np.array(p_values).reshape(-1, 1)
        feature_clusters = self.kmeans.fit_predict(p_values)
        self.sensor_cluster_mapping = dict(zip(sensor_ids, feature_clusters))
        
        cluster_labels = []
        for sensor_id in sensor_ids:
            cluster_idx = self.sensor_cluster_mapping[sensor_id]
            data_length = len(sensor_data_dict[sensor_id])
            cluster_labels.extend([cluster_idx] * data_length)
        
        return np.array(cluster_labels)
    
    def train_cluster_models(self, sensor_data, input_dim, device, weight_decay=1e-4):
        # 각 클러스터별 데이터 크기 파악
        cluster_sizes = []
        for cluster in range(self.n_clusters):
            cluster_data = sensor_data[sensor_data['cluster'] == cluster]
            cluster_sizes.append(len(cluster_data))
        
        max_size = max(cluster_sizes)

        for cluster in range(self.n_clusters):
            print(f"\nTraining model for cluster {cluster}")
            cluster_data = sensor_data[sensor_data['cluster'] == cluster]
            if len(cluster_data) == 0:
                continue

            # 센서 ID를 기준으로 데이터 정렬하여 시계열 연속성 유지
            cluster_data = cluster_data.sort_values(['sensor_id', 'timestamp'])
            numeric_data = cluster_data.drop(['cluster', 'sensor_id', 'timestamp'], axis=1).values

            # 일 단위 윈도우를 고려한 데이터 분할
            window_size = 1440  # 1일
            stride = 720       # 12시간
            
            total_windows = (len(numeric_data) - window_size) // stride + 1
            train_size = int(0.8 * total_windows)
            val_size = int(0.1 * total_windows)

            # 데이터셋 생성
            train_dataset = TimeSeriesSensorDataset(
                numeric_data[:train_size * stride + window_size],
                window_size=window_size,
                stride=stride
            )
            val_dataset = TimeSeriesSensorDataset(
                numeric_data[train_size * stride:train_size * stride + val_size * stride + window_size],
                window_size=window_size,
                stride=stride
            )
            test_dataset = TimeSeriesSensorDataset(
                numeric_data[train_size * stride + val_size * stride:],
                window_size=window_size,
                stride=stride
            )

            # 배치 크기 조정
            train_dataloader = DataLoader(
                train_dataset, 
                batch_size=64, 
                shuffle=False,
                num_workers=28,  # CPU 코어 수에 맞게 조정
                pin_memory=True  # GPU 메모리 전송 최적화
            )

            val_dataloader = DataLoader(
                val_dataset, 
                batch_size=64, 
                shuffle=False,
                num_workers=28,
                pin_memory=True
            )

            test_dataloader = DataLoader(
                test_dataset, 
                batch_size=64, 
                shuffle=False,
                num_workers=28,
                pin_memory=True
            )

            # 모델 초기화
            model = TimeSeriesSensorAutoencoder(
                input_dim=input_dim,
                window_size=window_size,
                weight_decay=weight_decay,
                sensor_weights=self.sensor_weights
            ).to(device)
            
            # Learning rate 감소 (시계열 학습 안정성을 위해)
            optimizer = Adam(model.parameters(), lr=0.001, weight_decay=weight_decay)

            best_val_loss = float('inf')
            patience = 12
            patience_counter = 0

            for epoch in range(70):
                model.train()
                total_train_loss = 0
                for windows, targets in tqdm(train_dataloader, desc=f"Cluster {cluster}, Epoch {epoch+1} [Train]", leave=False):
                    windows, targets = windows.to(device), targets.to(device)

                    optimizer.zero_grad()
                    z, reconstructed, total_loss = model(windows)

                    total_loss.backward()
                    optimizer.step()
                    total_train_loss += total_loss.item()

                avg_train_loss = total_train_loss / len(train_dataloader)

                model.eval()
                total_val_loss = 0
                with torch.no_grad():
                    for windows, targets in tqdm(val_dataloader, desc=f"Cluster {cluster}, Epoch {epoch+1} [Val]", leave=False):
                        windows, targets = windows.to(device), targets.to(device)
                        z, reconstructed, v_loss = model(windows)
                        total_val_loss += v_loss.item()

                avg_val_loss = total_val_loss / len(val_dataloader)

                if epoch % 5 == 0:
                    print(f"Cluster {cluster}, Epoch {epoch+1}, Train Loss: {avg_train_loss:.6f}, Val Loss: {avg_val_loss:.6f}")

                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    patience_counter = 0
                    best_model_state = model.state_dict()
                else:
                    patience_counter += 1

                if patience_counter >= patience:
                    print(f"Early stopping at epoch {epoch+1}")
                    break

            model.load_state_dict(best_model_state)

            # 테스트 단계
            reconstruction_errors = []
            model.eval()
            with torch.no_grad():
                for windows, targets in test_dataloader:
                    windows, targets = windows.to(device), targets.to(device)
                    z, reconstructed, _ = model(windows)
                    
                    # 마스크 생성 (마지막 시점에 대해서만)
                    q_mask = torch.zeros_like(targets)
                    q_mask[:, :MAX_Q_SENSORS] = (targets[:, :MAX_Q_SENSORS] != 0).float()
                    
                    m_mask = torch.zeros_like(targets)
                    m_mask[:, -2] = 1
                    
                    p_mask = torch.zeros_like(targets)
                    p_mask[:, -1] = 1
                    
                    combined_mask = q_mask + m_mask + p_mask
                    
                    errors = (torch.nn.functional.mse_loss(
                        reconstructed,
                        targets,
                        reduction='none'
                    ) * combined_mask).sum(dim=1) / combined_mask.sum(dim=1)

                    reconstruction_errors.extend(errors.cpu().numpy())

            # 통계 저장 및 임계값 계산
            self.cluster_stats[cluster] = {
                'threshold': np.percentile(reconstruction_errors, 95),  # 95 퍼센타일로 변경
                'mean_error': np.mean(reconstruction_errors),
                'std_error': np.std(reconstruction_errors),
                'best_val_loss': best_val_loss
            }

            self.cluster_models[cluster] = model

            print(f"\nCluster {cluster} Stats:")
            print(f"Mean Error: {self.cluster_stats[cluster]['mean_error']:.6f}")
            print(f"Threshold: {self.cluster_stats[cluster]['threshold']:.6f}")
            print(f"Best Validation Loss: {best_val_loss:.6f}")

In [ ]:
 
def prepare_data_by_sensor(path, max_q_sensors=MAX_Q_SENSORS):
    file_list = glob(os.path.join(path, '*.csv'))
    sensor_data = {}
    sensor_features = {}

    for file in tqdm(file_list, desc="Processing files", leave=False):
        df = pd.read_csv(file)
        df['timestamp'] = pd.to_datetime(df['timestamp'], format='%y/%m/%d %H:%M')
        
        padded_q_data, m_data = None, None
        p_sensors = [col for col in df.columns if col.startswith('P') and not col.endswith('_flag')]

        for p_sensor in p_sensors:
            if padded_q_data is None:
                padded_q_data, m_data, features = normalize_and_compute_features(
                    df, max_q_sensors
                )
            
            # timestamp를 제외한 숫자 데이터만 저장
            combined_data = np.concatenate([
                padded_q_data, 
                m_data.values[:, np.newaxis],
                df[p_sensor].values[:, np.newaxis]
            ], axis=1)
            
            sensor_id = f"{'TRAIN_A' if 'TRAIN_A' in file else 'TRAIN_B'}_{p_sensor}"
            if sensor_id not in sensor_data:
                sensor_data[sensor_id] = []
                sensor_features[sensor_id] = []
            
            # DataFrame으로 변환할 때 timestamp는 따로 저장
            data_df = pd.DataFrame(combined_data)
            data_df['timestamp'] = df['timestamp']  # timestamp를 마지막 열로 추가
            sensor_data[sensor_id].append(data_df)
            sensor_features[sensor_id].append(features)

    # 각 센서별로 시간순 정렬된 데이터 생성
    for sensor_id in sensor_data:
        sensor_data[sensor_id] = pd.concat(sensor_data[sensor_id], axis=0)
        sensor_data[sensor_id] = sensor_data[sensor_id].sort_values('timestamp')

    return sensor_data, sensor_features

In [ ]:
  
# 메인 실행 코드
print("Starting data processing...")

# 데이터 로드 및 전처리
sensor_data, sensor_features = prepare_data_by_sensor(train_path)

print("\nProcessing all sensors...")
cluster_manager = SensorClusterManager(n_clusters=6)

# 클러스터링 수행
cluster_labels = cluster_manager.compute_cluster_assignment(sensor_data)

# 데이터프레임 생성 및 클러스터 할당
all_data = []
for sensor_id in sensor_data:
    df = sensor_data[sensor_id].copy()
    df['sensor_id'] = sensor_id
    all_data.append(df)

df_combined = pd.concat(all_data, axis=0)
df_combined['cluster'] = cluster_labels

print("\nSensor Cluster Assignments:")
for sensor_id, cluster in cluster_manager.sensor_cluster_mapping.items():
    print(f"{sensor_id}: Cluster {cluster}")

# 모델 학습
cluster_manager.train_cluster_models(
    df_combined, 
    input_dim=MAX_Q_SENSORS + 2, 
    device=device  # window_size는 이미 train_cluster_models 안에서 1440으로 설정됨
)

# 결과 저장
torch.save({
    'cluster_models': cluster_manager.cluster_models,
    'cluster_stats': cluster_manager.cluster_stats,
    'kmeans': cluster_manager.kmeans,
    'sensor_cluster_mapping': cluster_manager.sensor_cluster_mapping
}, os.path.join(output_path, 'time_series_cluster_models.pth'))

In [ ]:
# 시각화 - 클러스터링 결과
plt.figure(figsize=(15, 5))
sensor_means = {}
for sensor_id in sensor_data:
    sensor_means[sensor_id] = np.mean(sensor_data[sensor_id].iloc[:, -1])

colors = plt.cm.get_cmap('tab20')(np.linspace(0, 1, cluster_manager.n_clusters))

for cluster in range(cluster_manager.n_clusters):
    cluster_sensors = [s for s, c in cluster_manager.sensor_cluster_mapping.items() if c == cluster]
    cluster_means = [sensor_means[s] for s in cluster_sensors]
    x_positions = range(len(cluster_sensors))
    plt.scatter(x_positions, 
                cluster_means,
                c=[colors[cluster]],
                label=f'Cluster {cluster}')
    plt.xticks(x_positions, cluster_sensors, rotation=45, ha='right')

plt.title('Average P Sensor Values by Cluster')
plt.ylabel('Normalized P Sensor Value')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# 클러스터별 시계열 패턴 시각화
for cluster in range(cluster_manager.n_clusters):
    cluster_data = df_combined[df_combined['cluster'] == cluster]
    print(f"\nCluster {cluster} contains sensors:")
    print(sorted(cluster_data['sensor_id'].unique()))
    
    plt.figure(figsize=(15, 12))
    
    # Q1 센서 시계열
    plt.subplot(3, 1, 1)
    for sensor_id in sorted(cluster_data['sensor_id'].unique()):
        sensor_data = cluster_data[cluster_data['sensor_id'] == sensor_id]
        plt.plot(sensor_data['timestamp'], sensor_data.iloc[:, 0], 
                label=sensor_id, alpha=0.5)
    plt.title(f'Q1 Sensor Time Series for Cluster {cluster}')
    plt.xlabel('Time')
    plt.ylabel('Q1 Value')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # M_sum 시계열
    plt.subplot(3, 1, 2)
    for sensor_id in sorted(cluster_data['sensor_id'].unique()):
        sensor_data = cluster_data[cluster_data['sensor_id'] == sensor_id]
        plt.plot(sensor_data['timestamp'], sensor_data.iloc[:, -2], 
                label=sensor_id, alpha=0.5)
    plt.title(f'M_sum Time Series for Cluster {cluster}')
    plt.xlabel('Time')
    plt.ylabel('M_sum Value')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    # P 센서 시계열
    plt.subplot(3, 1, 3)
    for sensor_id in sorted(cluster_data['sensor_id'].unique()):
        sensor_data = cluster_data[cluster_data['sensor_id'] == sensor_id]
        plt.plot(sensor_data['timestamp'], sensor_data.iloc[:, -1], 
                label=sensor_id, alpha=0.5)
    plt.title(f'P Sensor Time Series for Cluster {cluster}')
    plt.xlabel('Time')
    plt.ylabel('P Value')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # 재구성 오차 분포 시각화
    if cluster in cluster_manager.cluster_stats:
        plt.figure(figsize=(10, 5))
        threshold = cluster_manager.cluster_stats[cluster]['threshold']
        mean_error = cluster_manager.cluster_stats[cluster]['mean_error']
        plt.axvline(x=threshold, color='r', linestyle='--', label=f'Threshold: {threshold:.4f}')
        plt.axvline(x=mean_error, color='g', linestyle='--', label=f'Mean Error: {mean_error:.4f}')
        plt.title(f'Reconstruction Error Distribution for Cluster {cluster}')
        plt.xlabel('Reconstruction Error')
        plt.ylabel('Count')
        plt.legend()
        plt.tight_layout()
        plt.show()

print("\nTraining and visualization completed!")